In [ ]:
import pandas as pd

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import time
years = list(range(2014, 2006, -4)) #2022 and 2018
all_matches = []
standings_url = 'https://fbref.com/en/comps/1/2014/2014-World-Cup-Stats'

for year in years:
    try:
        print(f"Fetching standings for year {year} from URL: {standings_url}")
        data = requests.get(standings_url)
        data.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(data.text, 'html.parser')

        # Debugging: print the content of the page
        print(f"HTML content for year {year}: {data.text[:500]}")  # print the first 500 characters

        standings_table = soup.select(f"#results{year}10_overall")
        if not standings_table:
            print(f"No standings table found for year {year}")
            continue

        links = [l.get("href") for l in standings_table[0].find_all('a')]
        links = [l for l in links if '/squads/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]
        print(f"Found {len(team_urls)} team URLs")

        previous_competition = soup.select("a.prev")
        if not previous_competition:
            print(f"No previous competition link found for year {year}")
            continue

        standings_url = f"https://fbref.com{previous_competition[0].get('href')}"
        print(f"Next standings URL: {standings_url}")

        for team_url in team_urls:
            try:
                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
                print(f"Fetching data for team: {team_name} from URL: {team_url}")
                data = requests.get(team_url)
                data.raise_for_status()
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

                soup = BeautifulSoup(data.text, 'html.parser')
                links = [l.get("href") for l in soup.find_all('a')]
                links = [l for l in links if l and 'all_comps/shooting/' in l]
                if not links:
                    print(f"No shooting data found for team: {team_name}")
                    continue

                shooting_url = f"https://fbref.com{links[0]}"
                print(f"Fetching shooting data from URL: {shooting_url}")
                time.sleep(20)
                data = requests.get(shooting_url)
                data.raise_for_status()
                shooting = pd.read_html(data.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()

                try:
                  if year > 2014:
                        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
                  else:
                        if "FK" not in shooting.columns:
                            shooting["FK"] = 0
                        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
                except ValueError as e:
                    print(f"Error merging data for team: {team_name} - {e}")
                    continue

                team_data = team_data[team_data["Comp"] == "World Cup"]
                team_data["Season"] = year
                team_data["Team"] = team_name
                all_matches.append(team_data)

                print(f"Successfully fetched and processed data for team: {team_name}")

            except Exception as e:
                print(f"Error fetching data for team URL: {team_url} - {e}")

        print(f"Completed fetching data for year {year}")
        time.sleep(20)  # Delay between requests

    except Exception as e:
        print(f"Error fetching standings for year {year} - {e}")

print("Data collection completed.")


Fetching standings for year 2014 from URL: https://fbref.com/en/comps/1/2014/2014-World-Cup-Stats
HTML content for year 2014:     
      
<!DOCTYPE html>
<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >
<head>
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />
    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202407291" />
<script>
/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = w
Found 32 team URLs
Next standings URL: https://fbref.com/en/comps/1/2010/2010-World-Cup-Stats
Fetching data for team: Germany Men from URL: https://fbref.com/en/squads/c1e40422/2014/Germany-Men-Stats


<ipython-input-4-27c04770de7b>:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


Fetching shooting data from URL: https://fbref.com/en/squads/c1e40422/2014/matchlogs/all_comps/shooting/Germany-Men-Match-Logs-All-Competitions


<ipython-input-4-27c04770de7b>:54: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


Successfully fetched and processed data for team: Germany Men
Fetching data for team: Argentina Men from URL: https://fbref.com/en/squads/f9fddd6e/2014/Argentina-Men-Stats


...

Successfully fetched and processed data for team: Cameroon Men
Fetching data for team: Korea DPR Men from URL: https://fbref.com/en/squads/42c4106c/2010/Korea-DPR-Men-Stats


<ipython-input-4-27c04770de7b>:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


Fetching shooting data from URL: https://fbref.com/en/squads/42c4106c/2010/matchlogs/all_comps/shooting/Korea-DPR-Men-Match-Logs-All-Competitions


<ipython-input-4-27c04770de7b>:54: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


Successfully fetched and processed data for team: Korea DPR Men
Completed fetching data for year 2010
Data collection completed.


In [ ]:
all_matches

[          Date   Time       Comp           Round  Day    Venue Result  GF  GA  \
 14  2014-06-16  13:00  World Cup     Group stage  Mon  Neutral      W   4   0   
 15  2014-06-21  16:00  World Cup     Group stage  Sat  Neutral      D   2   2   
 16  2014-06-26  13:00  World Cup     Group stage  Thu  Neutral      W   1   0   
 17  2014-06-30  17:00  World Cup     Round of 16  Mon  Neutral      W   2   1   
 18  2014-07-04  13:00  World Cup  Quarter-finals  Fri  Neutral      W   1   0   
 19  2014-07-08  17:00  World Cup     Semi-finals  Tue  Neutral      W   7   1   
 20  2014-07-13  16:00  World Cup           Final  Sun  Neutral      W   1   0   
 
             Opponent  ...  Match Report                Notes  Sh SoT Dist FK  \
 14       pt Portugal  ...  Match Report                  NaN NaN NaN  NaN  0   
 15          gh Ghana  ...  Match Report                  NaN NaN NaN  NaN  0   
 16  us United States  ...  Match Report                  NaN NaN NaN  NaN  0   
 17        dz Alge

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
14,2014-06-16,13:00,World Cup,Group stage,Mon,Neutral,W,4,0,pt Portugal,...,Match Report,NaN,NaN,NaN,NaN,0,1,1,2014,Germany Men
15,2014-06-21,16:00,World Cup,Group stage,Sat,Neutral,D,2,2,gh Ghana,...,Match Report,NaN,NaN,NaN,NaN,0,0,0,2014,Germany Men
16,2014-06-26,13:00,World Cup,Group stage,Thu,Neutral,W,1,0,us United States,...,Match Report,NaN,NaN,NaN,NaN,0,0,0,2014,Germany Men
17,2014-06-30,17:00,World Cup,Round of 16,Mon,Neutral,W,2,1,dz Algeria,...,Match Report,Required Extra Time,NaN,NaN,NaN,0,0,0,2014,Germany Men
18,2014-07-04,13:00,World Cup,Quarter-finals,Fri,Neutral,W,1,0,fr France,...,Match Report,NaN,NaN,NaN,NaN,0,0,0,2014,Germany Men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2010-06-19,20:30,World Cup,Group stage,Sat,Neutral,L,1,2,dk Denmark,...,Match Report,NaN,NaN,NaN,NaN,0,0,0,2010,Cameroon Men
6,2010-06-24,20:30,World Cup,Group stage,Thu,Neutral,L,1,2,nl Netherlands,...,Match Report,NaN,NaN,NaN,NaN,0,1,1,2010,Cameroon Men
16,2010-06-15,20:30,World Cup,Group stage,Tue,Neutral,L,1,2,br Brazil,...,Match Report,NaN,NaN,NaN,NaN,0,0,0,2010,Korea DPR Men
17,2010-06-21,13:30,World Cup,Group stage,Mon,Neutral,L,0,7,pt Portugal,...,Match Report,NaN,NaN,NaN,NaN,0,0,0,2010,Korea DPR Men


In [ ]:
match_df.to_csv("matches.csv")